# Upsampling's tfgan experiment notebook



### Librerias

In [ ]:
from google.colab import drive
import zipfile
drive.mount('/content/drive')
with zipfile.ZipFile('/content/drive/MyDrive/TFG/UTK_Corto.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
experiment = Experiment(
  api_key = "g208he4vUVLoAY4sFmdPGDBDO",
  project_name = "Monitorización Final",
  workspace="kpstufforigins",
  log_code=True,
  log_graph=True,
)
experiment.add_tag("TFGAN Up 1")

In [ ]:
from scipy.stats import wasserstein_distance
import calendar
import time as t
import os
import numpy as np
import math
import sys
from time import time
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import log2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torchvision import transforms, utils
from torch.utils.tensorboard import SummaryWriter



from torch.utils.data import Dataset, DataLoader, random_split
import os
import pandas as pd
from tqdm import tqdm
import logging
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from math import log2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
LEARNING_RATE_GEN = 1e-4
LEARNING_RATE_DIS = 1e-4
LAMBDA_IDENTITY = 0
LAMBDA_CYCLE = 10
NUM_WORKERS = 2
NUM_EPOCHS = 1500
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_GEN_H = "genh.pth.tar"
CHECKPOINT_GEN_Z = "genz.pth.tar"
CHECKPOINT_CRITIC_H = "critich.pth.tar"
CHECKPOINT_CRITIC_Z = "criticz.pth.tar"

hyper_params = {
   "learning_rate_gen": LEARNING_RATE_GEN,
   "learning_rate_dis": LEARNING_RATE_DIS,
   "EPOCH": NUM_EPOCHS,
   "batch_size": BATCH_SIZE,
   "lambda_identity": LAMBDA_IDENTITY
}

### ARCH

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), # Pads the input tensor using the reflection of the input boundary
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.block(x)



class Generator_TFGan_Up_Modelo_1(nn.Module):

    #Model 1 do not downsample the image, it multiplex the channels
    #then apply the residuals blocks and directly upsample

    def __init__(self, img_resolution, init_channels=3, frontier=3, num_residuals=3, rabit_hole=2):
        """
        -> frontier es el numero de multiplexaciones iniciales
        -> num_residuals es el numero de bloques residuales
        -> rabit_hole es cuantos pasos de reescalamiento hay al final

        Por default las imagenes se reescalan 2 veces, es decir, la resolucion final sera 4*img_resolution
        """

        super().__init__()
        self.img_resolution = img_resolution
        self.frontier = frontier
        self.num_residuals = num_residuals
        current_size = img_resolution
        self.init_channels = init_channels

        #Initial Layers, Extraordinary upsampling
        out_features = 64
        self.model = [
            nn.Conv2d(init_channels, out_features, kernel_size=3,stride=1,padding=1),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True)
        ]
        in_features = out_features

        #Llegada a la frontera
        for _ in range(frontier):
            out_features *= 2
            self.model += [
              nn.Conv2d(in_features, out_features, 3, stride=1, padding=1), # --> width and height constant
              nn.InstanceNorm2d(out_features),
              nn.ReLU(inplace=True)
            ]
            in_features = out_features

        #RESIDUAL
        for _ in range(num_residuals):
            self.model += [ResidualBlock(out_features)]

        #UPSAMPLING
        for _ in range(rabit_hole):
            out_features //= 2
            self.model += [
                nn.Upsample(scale_factor=2), # --> width*2, heigh*2
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            current_size *= 2

        # Output Layer
        self.model += [
                  nn.Conv2d(out_features, init_channels, 3,stride=1,padding=1),
                  nn.Tanh()
                 ]

        #UNPACK
        self.model_sequential = nn.Sequential(*self.model)

        return

    def forward(self,x):
        #print(f"init shape{x.shape}")
        for layer in self.model:
            x = layer(x)
        #print(f"for layer:{layer} shape returned is: {x.shape}")
        return x
        #return self.model_sequential(x)

def test():
    x = torch.randn((2,3,128,128))
    model = Generator_TFGan_Up_Modelo_1(128)
    #print(model)
    print(f"shape returned by model: {model(x).shape}")
    return

test()

shape returned by model: torch.Size([2, 3, 512, 512])


In [ ]:
class Discriminador(nn.Module):
    def __init__(self, channels = 3):
        super().__init__()
        self.channels = channels
        nc = channels
        ndf = 64
        self.layers = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(nc,ndf,4,2,1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 64 x 64
            nn.Conv2d(ndf,ndf*2,4,2,1, bias=False),
            nn.InstanceNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 32 x 32
            nn.Conv2d(ndf*2, ndf * 4, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            #ResidualBlock(ndf*4),
            #ResidualBlock(ndf*4),

            # state size. (ndf*4) x 16 x 16
            nn.Conv2d(ndf*4,ndf*8,4,1,1),
            nn.InstanceNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 15 x 15
            nn.Conv2d(ndf*8,1,4,1,1)
            # state size. 1 x 14 x 14
        )

        return


    def forward(self, x):
      #print(f"input shape: {x.shape}")
      return self.layers(x)

def test(i):
    img_size = i
    model = Discriminador().to(DEVICE)

    print(f"img_size: {img_size}")
    x = torch.randn((1,3,img_size,img_size)).to(DEVICE)

    print(model(x))
    #print(torch.mean(model(x)))
    print(model(x).shape)
    #print(model)
    return

test(64)


img_size: 64
tensor([[[[-0.2721, -0.0638,  0.6127, -0.7428,  0.1489, -0.0569],
          [ 0.4339,  0.3629, -0.3181,  0.0514,  0.1529, -0.1327],
          [-0.2702, -0.4321, -0.4469, -0.4032, -0.5285,  0.5252],
          [ 0.1467, -0.3046,  0.2535, -0.0860, -0.7569, -0.6893],
          [ 0.2383,  0.5676,  0.4166, -1.0149, -0.0142, -0.0887],
          [-0.1984, -0.1738,  0.7834, -0.2528, -0.1990,  0.0903]]]],
       device='cuda:0', grad_fn=<ConvolutionBackward0>)
torch.Size([1, 1, 6, 6])


### Data

In [ ]:
transforms1 = A.Compose(
    [
        A.Resize(width=32, height=32),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5 for _ in range(3)], std=[0.5 for _ in range(3)], max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"},
)

transforms2 = A.Compose(
    [
        A.Resize(width=16, height=16),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5 for _ in range(3)], std=[0.5 for _ in range(3)], max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"},
)

In [ ]:
from PIL import Image
import os
from torch.utils.data import Dataset
import numpy as np

class TFGAN_Dataset(Dataset):

    #Por estandar
    #  root_A = Adultos
    #  root_b = Kids

    def __init__(self, root_A, root_B, transform1=None, transform2=None, gray_scale=True):
        self.root_A = root_A
        self.root_B = root_B
        self.transform1 = transform1
        self.transform2 = transform2
        self.gray_scale = gray_scale

        self.root_A_images = os.listdir(root_A)
        self.root_B_images = os.listdir(root_B)
        self.length_dataset = max(len(self.root_A_images), len(self.root_B_images))
        self.root_A_len = len(self.root_A_images)
        self.root_B_len = len(self.root_B_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):

        A_image = self.root_A_images[index % self.root_A_len]
        B_image = self.root_B_images[index % self.root_B_len]

        A_path = os.path.join(self.root_A, A_image)
        B_path = os.path.join(self.root_B, B_image)

        A_image = np.array(Image.open(A_path).convert("L" if self.gray_scale else "RGB"))
        B_image = np.array(Image.open(B_path).convert("L" if self.gray_scale else "RGB"))

        A_image1 = A_image
        A_image2 = A_image

        B_image1 = B_image
        B_image2 = B_image

        if self.transform1:
            augmentations1 = self.transform1(image=A_image, image0=B_image)
            augmentations2 = self.transform2(image=A_image, image0=B_image)

            A_image1 = augmentations1["image"]
            A_image2 = augmentations2["image"]
            B_image1 = augmentations1["image0"]
            B_image2 = augmentations2["image0"]


        return A_image1, B_image1, A_image2, B_image2

### Train

In [ ]:
def loss_gan(fake,real):
    MSE = nn.L1Loss()
    return MSE(fake,real)

def loss_dis(fake, real):
    MSE = nn.MSELoss()
    return MSE(real,fake)

def loss_id(in1, out1, in2, out2):
    return torch.abs(in1-out1).mean() + torch.abs(in2-out2).mean()

def dis_loss_wasserstein_gan(fake_img, real_img):
    return torch.mean(real_img)-torch.mean(fake_img)

def gen_loss_wasserstein_gan(fake_img): #fake_img es gen(input)
    return torch.mean(fake_img)


In [ ]:
### Checkpoints
import random, torch, os, numpy as np
import copy

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    #print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
import torch
import sys
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image

def train_individual_down(
    writer, epoch, i,
    loader, l1, mse,
    disc_Adults_down, disc_Kids_down, down_sampler, opt_disc_Adults_down, opt_disc_Kids_down, opt_gen_down, d_down_scaler, g_down_scaler,
):

    adults_reals = 0
    adults_fakes = 0

    loop = tqdm(loader, leave=True)

    for idx, (adult, kid, adult_frontier, kid_frontier) in enumerate(loop):
        i+=1
        #Mandamos las cosas a su sitio
        adult = adult.to(DEVICE)
        adult_frontier = adult_frontier.to(DEVICE)
        kid = kid.to(DEVICE)
        kid_frontier = kid_frontier.to(DEVICE)

        #Entrenamos los discriminadores
        with torch.cuda.amp.autocast():
            fake_adult_down = down_sampler(kid)

            #DOWN
            disc_adult_down_real = disc_Adults_down(adult_frontier)
            disc_adult_down_fake = disc_Adults_down(fake_adult_down.detach())

            disc_adult_down_loss = torch.mean(disc_adult_down_real) - torch.mean(disc_adult_down_fake)
            writer.add_scalar("DisLoss", disc_adult_down_loss,  i)




            #Como no hay consistencia de ciclo no neecesitamos entrenar los segundos generadores
            #Lo juntamos todo, no nos sirve para entreno solo para ver como va la cosa

            D_loss = disc_adult_down_loss


        #APLICAMOS LOS CAMBIOS DEL ENTRENO:

        opt_disc_Adults_down.zero_grad()
        d_down_scaler.scale(D_loss).backward()
        d_down_scaler.step(opt_disc_Adults_down)
        d_down_scaler.update()


        #Ahora entrenamos el generador  con lo que hemos sacado de los discriminadores
        with torch.cuda.amp.autocast():

            generated_images = down_sampler(kid)

            #adversarial loss
            dis_adult_down_fake = disc_Adults_down(generated_images)
            loss_gen_down_adult = torch.mean(dis_adult_down_fake)
            writer.add_scalar("GenLoss", loss_gen_down_adult, i)


            #Se suma todo
            g_down_loss = loss_gen_down_adult

        #APLICAMOS LOS CAMBIOS DEL ENTRENO:

        #DOWN
        opt_gen_down.zero_grad()
        g_down_scaler.scale(g_down_loss).backward()
        g_down_scaler.step(opt_gen_down)
        g_down_scaler.update()


        if idx==0:
            timestamp = t.time()
            save_image(generated_images*0.5+0.5,f"/content/drive/MyDrive/TFG/UpSamplerModelo1/output{timestamp}.png")
            save_image(kid*0.5+0.5,f"/content/drive/MyDrive/TFG/UpSamplerModelo1/input{timestamp}.png")
            save_checkpoint(down_sampler, opt_gen_down, filename="/content/drive/MyDrive/TFG/UpSamplerModelo1/gen_down.pth.tar")
            save_checkpoint(disc_Kids_down, opt_disc_Adults_down, filename="/content/drive/MyDrive/TFG/UpSamplerModelo1/disc_Kids_down.pth.tar")
            save_checkpoint(disc_Adults_down, opt_disc_Kids_down, filename="/content/drive/MyDrive/TFG/UpSamplerModelo1/disc_Adults_down.pth.tar")


            img_grid = torchvision.utils.make_grid(adult)
            #matplotlib_imshow(img_grid, one_channel=True)

            writer.add_image('input_down', torchvision.utils.make_grid(kid), i)

            writer.add_image('output_down', torchvision.utils.make_grid(fake_adult_down), i)
            #writer.add_image('fake_adults_down', torchvision.utils.make_grid(fake_adult_down),idx)
            #writer.add_image('fake_adults_res', torchvision.utils.make_grid(fake_adult_res), idx)
            #writer.add_graph(down_sampler,kid)

        loop.set_postfix(GenLoss = torch.mean(g_down_loss), DisLoss = torch.mean(D_loss))



### Main

In [ ]:
def main():


    #Dataset
    img_size = 64
    img_size_discriminator = 64*4
    channels = 3

    dataset_full = A.Compose([
        A.Resize(width=img_size, height=img_size),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5 for _ in range(channels)], std=[0.5 for _ in range(channels)], max_pixel_value=255),
        ToTensorV2(),
    ],additional_targets={"image0": "image"},)

    dataset_discriminator = A.Compose([
            A.Resize(width=img_size_discriminator, height=img_size_discriminator),
            A.HorizontalFlip(p=0.5),
            A.Normalize(mean=[0.5 for _ in range(channels)], std=[0.5 for _ in range(channels)], max_pixel_value=255),
            ToTensorV2(),
        ],additional_targets={"image0": "image"},)


    train_path = "."
    Adults_train = os.path.join(train_path, 'Kids')
    Kids_train = os.path.join(train_path, 'Adults')

    dataset = TFGAN_Dataset(Adults_train, Kids_train, transform1=dataset_full, transform2=dataset_discriminator,gray_scale=False)
    loader = DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=NUM_WORKERS,pin_memory=True,)

    #Loss functions
    L1 = nn.L1Loss()
    mse = nn.MSELoss()



    disc_Adults_down = Discriminador().to(DEVICE)
    disc_Kids_down = Discriminador().to(DEVICE)
    down_sampler = Generator_TFGan_Up_Modelo_1(img_size).to(DEVICE)

    opt_disc_Adults_down = optim.Adam(disc_Adults_down.parameters(),
                               lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
    opt_disc_Kids_down = optim.Adam(disc_Kids_down.parameters(),
                               lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
    opt_gen_down = optim.Adam(down_sampler.parameters(),
                              lr=LEARNING_RATE_GEN,betas=(0.5, 0.999),)
    d_down_scaler = torch.cuda.amp.GradScaler()
    g_down_scaler = torch.cuda.amp.GradScaler()





    #debug
    print(down_sampler)
    x = torch.randn((1,channels,img_size,img_size)).to(DEVICE)
    print(down_sampler(x).shape)
    #TensorBoard
    writer = SummaryWriter()
    i = 0
    for epoch in range(NUM_EPOCHS):

        print(f"Currently in epoch: {epoch+1}")
        writer.add_scalar("Epoch", epoch, i)
        train_individual_down(
            writer, epoch, i,
            loader, L1, mse,
            disc_Adults_down, disc_Kids_down, down_sampler,
            opt_disc_Adults_down, opt_disc_Kids_down,
            opt_gen_down, d_down_scaler, g_down_scaler,
        )
        i+=1
    writer.close()

    return

if __name__ == "__main__":
    main()